In [150]:
import jieba
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [151]:
from os import listdir
from os.path import isfile, join,splitext
t_dir = 'dataTrainComplete'
txt_fnames = [splitext(f)[0] for f in listdir(t_dir) if isfile(join(t_dir, f))]

In [152]:
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()

pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()

In [153]:
from itertools import chain
vector_dict = {}
for idx,line in enumerate(chain(crop_line_sep,pest_line_sep,chem_line_sep)):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[word] = idx

In [154]:
vectors = {}
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [0]*764
    for seg in seg_list:
        if(seg in vector_dict):
            vectors[fname][vector_dict[seg]] = 1

In [155]:
labels = {}
pair_list = []
for fname1 in txt_fnames:
    for fname2 in txt_fnames:
        if(fname1 == fname2):continue
        else:
            labels[(fname1,fname2)] = 0
            pair_list.append((fname1,fname2))
corr_list = open('TrainLabel.csv', "r",encoding='UTF-8-sig')
corr = corr_list.read()
corr_line_sep = corr.splitlines()
for line in corr_line_sep[1:]:
    l = line.split(',')
    labels[(l[0],l[1])] = 1
print(len(labels))
print(sum(labels.values()))
print(len(pair_list))

313040
1383
313040


In [156]:
# Dataset
class PartDataset(data.Dataset):
    
    def __init__(self, labels, pair_list, vectors, phase='train'):
        self.labels = labels
        self.pair_list = pair_list
        self.vectors = vectors
        self.phase = phase
        
    def __len__(self):
        return len(self.pair_list)
    
    def __getitem__(self, idx):
        Test,Ref = pair_list[idx]
        label = self.labels[(Test,Ref)]
        comb_vector = self.vectors[Test] + self.vectors[Ref]
        return torch.tensor(comb_vector), label

In [157]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [158]:
train_dataset = PartDataset(labels, txt_fnames, vectors,phase='train')
test_dataset = PartDataset(labels, txt_fnames, vectors,phase='test')

In [159]:
print('Operation Check')
print(labels[('3','415')])
print(pair_list.index(('3','415')))
print(train_dataset.__getitem__(133886)[1])

Operation Check
1
133886
1


In [161]:
# DataLoader
train_dataloader = data.DataLoader(train_dataset, batch_size=4000, shuffle=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=4000, shuffle=False)

dataloader_dict = {'train': train_dataloader, 'test': test_dataloader}

# Operation Check
print('Operation Check')
batch_iterator = iter(train_dataloader)
inputs, label = next(batch_iterator)
print(len(label))

Operation Check
560


In [162]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2*764, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [163]:
net = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=net.parameters(), lr=0.001, momentum=0.9)

In [164]:
import time
def train_model(net, dataloader_dict, criterion, optimizer, num_epoch):
    
    since = time.time()
    best_model_wts = copy.deepcopy(net.state_dict())
    best_acc = 0.0
    net = net.to(device)
    
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
        
        for phase in ['train', 'test']:
            
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.type(torch.FloatTensor).to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(net.state_dict())
                print(time.time())
                torch.save(net.state_dict(), 'best_checkpoint_last.pth')
                
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    net.load_state_dict(best_model_wts)
    return net

In [165]:
num_epoch = 100
net = train_model(net, dataloader_dict, criterion, optimizer, num_epoch)

Epoch 1/100
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

train Loss: 0.6876 Acc: 0.9964


  0%|          | 0/1 [00:00<?, ?it/s]

test Loss: 0.6869 Acc: 1.0000
1638443803.2066517
Epoch 2/100
--------------------


...

test Loss: 0.3059 Acc: 1.0000
Epoch 100/100
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

train Loss: 0.3059 Acc: 1.0000


  0%|          | 0/1 [00:00<?, ?it/s]

test Loss: 0.3035 Acc: 1.0000
Training complete in 0m 19s
Best val Acc: 1.000000
